## CLEAN CARS

In [25]:
from common_functions import *
import os
os.chdir("../..")
from sibr_module import Logger, CStorage
from src.helper_modules import CustomBigQuery
from sibr_market_training import Clean
import re
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('future.no_silent_downcasting', True)
from typing import List
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
replace = True
dataset = 'cars'
logger = Logger(f'clean{dataset.capitalize()}')
logger.log_level = "DEBUG"
bq = CustomBigQuery(logger=logger,dataset=dataset)
cs = CStorage(logger=logger, bucket_name='sibr-market')
self = Clean(logger=logger, dataset=dataset)
logger.debug(f'Replace: {replace} | Dataset: {dataset}')

2025-08-27 10:43:07,298 - cleanCars - INFO - Cloud Logging is disabled. Using local logging to /Users/sigvardbratlie/Documents/Projects/sibr_market_training/logfiles/cleanCars.log.
2025-08-27 10:43:07,302 - cleanCars - INFO - BigQuery client initialized with project_id: sibr-market
2025-08-27 10:43:07,304 - cleanCars - INFO - Google Cloud Storage client initialized with bucket: sibr-market
2025-08-27 10:43:07,306 - cleanCars - INFO - BigQuery client initialized with project_id: sibr-market
2025-08-27 10:43:07,308 - cleanCars - INFO - Google Cloud Storage client initialized with bucket: sibr-market
2025-08-27 10:43:07,309 - cleanCars - DEBUG - Dataset: cars | | Replace: False
2025-08-27 10:43:07,310 - cleanCars - INFO - Clean class initialized.
2025-08-27 10:43:07,311 - cleanCars - DEBUG - Replace: True | Dataset: cars


In [31]:
dataset

'cars'

In [32]:
df_ = bq.read_raw(replace = replace,dataset=dataset)
geo = bq.read_geonorge()

2025-08-27 10:43:09,548 - cleanCars - INFO - Reading raw data from dataset: raw.cars
2025-08-27 10:44:32,383 - cleanCars - INFO - 810735 rader lest fra BigQuery
2025-08-27 10:44:34,246 - cleanCars - INFO - 5137 rader lest fra BigQuery


## Remove empty feature and missing data

In [33]:
self.df = df_
df = self.transform_nan(df_)
df = self.rm_empty_features(df)
self.logger.debug(f'Length: {len(df)} | after merge with sales time')

2025-08-27 10:44:34,307 - cleanCars - DEBUG - Length of df before cleaning: 810735
2025-08-27 10:44:49,490 - cleanCars - DEBUG - Length: 810735 | after removing columns with >90% missing values
2025-08-27 10:44:49,490 - cleanCars - DEBUG - Length: 810735 | after merge with sales time


## Int and Float data

In [17]:
import re
import ast
def extract_int(x):
    """
    Extracts an integer from a string.

    Args:
        x (str): The input value from which to extract the number.
    Returns:
        int or None: The extracted integer, or None if no valid number can be extracted.

    Examples
        "200.000" : str -> 200000 : int
        "200,000" : str -> 200000 : int
        "200m2" : str -> 200 : int
        "2000,0 : str -> 2000000 : int
        "435,5 kr : srt -> 435500 : int
    """
    if not isinstance(x, str):
        return None
    uten_mellomrom = re.sub(r'\s', '', x)
    match = re.search(r'[\d.,]+', uten_mellomrom)
    if not match:
        return None
    nummer_str = match.group(0)
    has_separator = '.' in nummer_str or ',' in nummer_str
    # Parse to float using similar logic as extract_float
    if ',' in nummer_str and '.' in nummer_str:
        if nummer_str.rfind(',') > nummer_str.rfind('.'):
            nummer_str_parsed = nummer_str.replace('.', '').replace(',', '.')
        else:
            nummer_str_parsed = nummer_str.replace(',', '')
    elif ',' in nummer_str:
        nummer_str_parsed = nummer_str.replace(',', '.')
    else:
        nummer_str_parsed = nummer_str
    try:
        nummer_float = float(nummer_str_parsed)
    except (ValueError, TypeError):
        return None
    if not has_separator:
        if nummer_float.is_integer():
            return int(nummer_float)
        else:
            return None
    else:
        # Multiply by 1000 and round to int
        multiplied = nummer_float * 1000
        return int(round(multiplied))

def extract_float(x):
    """
    Extracts  a float number from a string.

    Args:
        x (str): The input value from which to extract the number.
    Returns:
        float or None: The extracted floating number, or None if no valid number can be extracted.

    Examples
        "200.000" : str -> 200.0 : float
        "200,000" : str -> 200.0 : float
        "200m2" : str -> 200.0 : float
        "2000,0 : str -> 2000.0 : float
        "435,5 kr : srt -> 435.5 : float
            """
    if not isinstance(x, str):
        return None
    uten_mellomrom = re.sub(r'\s', '', x)
    match = re.search(r'[\d.,]+', uten_mellomrom)
    if not match:
        return None
    nummer_str = match.group(0)
    if ',' in nummer_str and '.' in nummer_str:
        if nummer_str.rfind(',') > nummer_str.rfind('.'):
            nummer_str = nummer_str.replace('.', '').replace(',', '.')
        else:
            nummer_str = nummer_str.replace(',', '')
    elif ',' in nummer_str:
        nummer_str = nummer_str.replace(',', '.')
    try:
        return float(nummer_str)
    except (ValueError, TypeError):
        return None

In [29]:
df_

NameError: name 'df_' is not defined

In [24]:
extract_int(s[-1]),extract_float(s)

(76, None)

In [10]:
int_cols = [
    'model_year', 'mileage', 'transfer_fee', 'price_excl_transfer', 'power',
    'co2', 'weight', 'seats', 'prev_owners', 'doors', 'trailer_weight',
    'last_eu', 'next_eu', 'range', 'battery', 'liens', 'total_price', 'rn'
]
float_cols = ['engine_volume', 'cargo_space']

df = self.mk_num(df, int_cols,type='int')
df = self.mk_num(df, float_cols,type='float')

2025-08-14 14:34:18,354 - cleanCars - DEBUG - Length: 595805 | after filter price and usable_area


In [ ]:
df['scrape_date'] = pd.to_datetime(df['scrape_date'],errors='coerce',utc=True)
df['clean_date'] = pd.Timestamp.now()
logger.debug(f'Length: {len(df)} | after datetime conversion')

## Geographical data

In [12]:
df.loc[:,'postal_code'] = df['address'].apply(extract_postnummer)
df = pd.merge(df,geo[['postal_code','municipality','county','region']],how='left',on='postal_code')
logger.debug(f'Length: {len(df)} | after geo')

2025-08-14 14:34:19,487 - cleanCars - DEBUG - Length: 595805 | after geo


## Categorical Data

In [13]:
fuel_mapping = {
    'diesel': 'diesel',
    'bensin': 'bensin',
    'el': 'el',
    'elektrisitet': 'el',
    'el + bensin': 'el + bensin',
    'hybrid bensin': 'el + bensin',
    'plug-in bensin': 'el + bensin',
    'el + diesel': 'el + diesel',
    'hybrid diesel': 'el + diesel',
    'plug-in diesel': 'el + diesel',
    'hydrogen': 'hydrogen'
}
brand_mapping = {'tesla motors':'tesla',
             'bmw i':'bmw',
             'alfa' : 'alfa romeo',
             'jaguar land rover limited' : 'land rover',
             'automobili lamborghini s.p.a.' : 'lamborghini',
             'land' : 'land rover',
             'rover' : 'land rover',
             'range rover' : 'land rover',
             'mercedes sprinter / kegger' : 'mercedes-benz',
             'mercedes-amg' : 'mercedes-benz',
             'ford-cng-technik' : 'ford',
             'daimler' : 'mercedes-benz',
             'kg mobility' : 'kgm',
             'mitsubishi fuso' : 'mitsubishi',
             'jaguar cars limited' : 'jaguar'}

for col in ['fuel', 'brand']:
    #logger.debug(f'Processing column: {col}')
    df.loc[:,col] = df[col].str.lower()
    if col == 'fuel':
        df.loc[:,col] = df[col].map(fuel_mapping).fillna('annet')
    elif col == 'brand':
        df.loc[:,col] = df[col].replace(brand_mapping)

In [14]:
#Fill missing values
df = fill_na(df,feature='gearbox', fill_value='Automat')
df = fill_na(df,feature='body_type', fill_value='SUV/Offroad')
df = fill_na(df,feature='sales_type', fill_value='Bruktbil til salgs')
df = fill_na(df,feature='dealer', fill_value='private')
df = fill_na(df,feature='known_issues', fill_value='False')
df = fill_na(df,feature='major_repairs', fill_value='False')
df = fill_na(df,feature='engine_tuned', fill_value='False')
df = fill_na(df,feature='liens', fill_value='False')
df['liens'] = df['liens'].astype('object').astype(str)

/Users/sigvardbratlie/Documents/Projects/sibr_market_training/src/cleaning/common_functions.py:114: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['False' 'False' 'False' ... 'False' 'False' 'False']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:,feature] = df[feature].fillna(fill_value)


In [15]:
# Categorical features from description
keys_carparts = ['delebil', 'rep objekt', 'reparasjonsobjekt', 'repobjekt', 'restaureringsobjekt',
                   'reparasjons objekt', 'motorhavari', 'motor havari','motorhavart',
                   'motor starter ikke', 'motoren starter ikke', 'motorhavarert', 'motor havarert',
                   'motorfeil', 'motor feil', 'chippet', 'defekt motor', 'rådebank']
keys_regreim = ['regreim', 'reg.reim', 'reg reim', 'reg.reim byttet', 'registerreim',
                'registerreim byttet',]

df = mk_bool_description(df,'car_for_parts',keys_carparts)
df = mk_bool_description(df,'timing_belt',keys_regreim)

logger.debug(f'Length: {len(df)} | after categorical')

2025-08-14 14:35:53,452 - cleanCars - DEBUG - Length: 595805 | after categorical


In [16]:
equipment_features = {
    'eq_cruise_control': ['cruisekontroll','cruisekontroll adaptiv','cruise controll adaptive','adaptiv cruisekontroll','adaptive cruise control','adaptiv cruise control'],
    'eq_parking_sensor_behind': ['parkeringsensor bak','avstandsføler bak','parkeringssensorer: bak','parkeringssensor bak'],
    'eq_parking_sensor_front': ['parkeringsensor foran','avstandsføler foran','parkeringssensorer: foran','parkeringssensor foran'],
    'eq_parking_assistant': ['parkeringsassistent'],
    'eq_driving_computer': ['kjørecomputer'],
    'eq_rear_view_camera': ['ryggekamera'],
    'eq_bluetooth': ['bluetooth','bluetooth: mobil','bluetooth handsfree','bluetooth®-forbindelse','bluetooth telefon og streaming'],
    'eq_tow_hitch': ['hengerfeste','henger feste','avtagbart hengerfeste','tilhengerfeste'],
    'eq_360_camera': ['360 kamera','360-kamera','360 graders overvåkning'],
    'eq_internet': ['internett','internet','internettilkobling'],
    'eq_app': ['app-tilkobling','app'],

    'eq_skin_interior': ['skinninteriør','skinn seter', 'skinnseter','skinn','seter i helskinn','skinn'],
    'eq_air_conditioning': ['klimaanlegg', 'aircondition','air condition','klimaanlegg bak','klimaanlegg foran','automatisk klimaanlegg','ventilasjon'],
    'eq_dab': ['dab-radio','dab radio','dab'],
    'eq_apple_carplay': ['apple carplay'],
    'eq_TCS': ['antispinn'],
    'eq_ESC': ['antiskrens'],
    'eq_sunroof': ['soltak','takluke','soltak/glasstak','panorama glasstak'],
    'eq_traffic_sign_recognition': ['trafikkskiltgjenkjenning','veiskiltinformasjon'],

    'eq_led_lights': ['led hovedlys','ledlys','led lys','lys led kjørelys'],
    'eq_xenon_lights': ['xenonlys','bi-xenon','xenon'],
    'eq_alarm': ['alarm','tyverialarm'],
    'eq_high_beam_assistant': ['fjernlysassistent'],

    'eq_keyless_start': ['nøkkelløs start','nøkkelfri lås/start'],
    'eq_keyless_go': ['nøkkelløs adgang','keyless go','nøkkelløs åpning'],

    'eq_abs': ['abs-bremser','abs'],
    'eq_isofix': ['isofix','isofix barnesetefesting','isofix: bak','isofix-monteringspunkter'],
    'eq_roof_railing': ['takrails','takstativ','takreling'],
    'eq_roof_box': ['takboks'],
    'eq_pollen_filter': ['pollenfilter','pollen filter'],
    'eq_airbag': ['airbags','kollisjonspute', 'kollisjonsputer','airbag foran side','airbag bak side'],
    'eq_power_steering': ['servostyring'],
    'eq_metallic_paint': ['metallisk lakk', 'metallakk'],
    'eq_handsfree': ['handsfree','handsfree opplegg','bluetooth handsfree'],
    'eq_alloy_wheels_summer': ['lettmet. felg sommer'],
    'eq_alloy_wheels_winter': ['lettmet. felg vinter'],
    'eq_navigation': ['navigasjonssystem'],

    'eq_charging_cable': ['laderkabel','lader kabel','ladekabel','ladeledning'],

    'eq_central_locking': ['sentrallås','fjernstyrt sentrallås'],
    'eq_rain_sensor': ['regnsensor'],

    'eq_heated_seats': ['oppvarmede seter','oppvarmede seter foran', 'oppvarmede seter bak','oppvarmede forseter','oppvarmede bakseter','varme i seter','setevarme foran','setevarme bak','setevarmer foran','setevarmer bak'],
    'eq_heated_steering_wheel': ['oppvarmet ratt', 'varmeratt','ratt oppvarmet'],
    'eq_electric_tailgate': ['elektrisk bakluke', 'el. bakluke', 'el bakluke', 'elektrisk bakdør'],
    'eq_electric_mirrors': ['el.mirroer', 'el speil', 'elektriske speil'],
    'eq_electric_windows': ['el.vinduer', 'el vinduer', 'elektriske vinduer','vindusheiser elektriske foran og bak'],
    'eq_electric_seats': ['elektrisk seteregulering foran', 'elektrisk seteregulering bak',
                        'el. seter foran','el. seter bak','elektriske seter','elektrisk sete'],
    'eq_heated_windshield': ['oppvarmet frontrute', 'oppvarmet vindu', 'oppvarmet vindusvisker'],

    'eq_winter_tires': ['vinterhjul','vinterdekk'],
    'eq_summer_tires': ['sommerhjul','sommerdekk'],
    'eq_multi_function_stearing_wheel': ['multifunksjonsratt'],
    'eq_block_heater': ['motorvarmer']
}
df = mk_bool_features(df, equipment_features, source_col='features')
logger.debug(f'Length: {len(df)} | after equipment features')

2025-08-14 14:37:48,538 - cleanCars - DEBUG - Length: 595805 | after equipment features


## Dates

In [34]:
import re
from datetime import datetime
from dateutil import parser

In [35]:
def extract_datetime(x):
    """
    Extracts a datetime object from a string, handling both English and Norwegian month names.

    Args:
        x (str): The input string containing the date and time.
    Returns:
        datetime or None: The extracted datetime, or None if parsing fails.

    Examples:
        'July 12, 2025, at 12:23\u202fPM' -> datetime(2025, 7, 12, 12, 23)
        '2024-05-28 13:35:00' -> datetime(2024, 5, 28, 13, 35)
        '4. februar 2025, 17:12' -> datetime(2025, 2, 4, 17, 12)
        'August 23, 2025, at 06:08\u202fAM' -> datetime(2025, 8, 23, 6, 8)
    """
    if not isinstance(x, str):
        return None
    x = x.replace('\u202f', ' ')
    nor_to_eng = {
        'januar': 'January',
        'februar': 'February',
        'mars': 'March',
        'april': 'April',
        'mai': 'May',
        'juni': 'June',
        'juli': 'July',
        'august': 'August',
        'september': 'September',
        'oktober': 'October',
        'november': 'November',
        'desember': 'December',
    }
    x_trans = x
    for nor, eng in nor_to_eng.items():
        x_trans = re.sub(r'\b' + re.escape(nor) + r'\b', eng, x_trans, flags=re.IGNORECASE)
    try:
        dt = parser.parse(x_trans, fuzzy=True)
        return dt
    except (ValueError, TypeError):
        return None

In [38]:
string_ = "6.5.2025"
extract_datetime(string_)

datetime.datetime(2025, 6, 5, 0, 0)

In [41]:
df["last_eu"] = df["last_eu"].apply(lambda x: extract_datetime(x) if isinstance(x, str) else x)
df["last_eu"]

0                         NaN
1                         NaN
2         2025-04-14 00:00:00
3                         NaN
4                         NaN
                 ...         
810730                    NaN
810731                    NaN
810732                    NaN
810733                    NaN
810734                    NaN
Name: last_eu, Length: 810735, dtype: object

In [17]:
df.loc[:,'FIRST'] = pd.to_datetime(df['FIRST'], errors='coerce', utc=True)
df.loc[:,'LAST'] = pd.to_datetime(df['LAST'], errors='coerce', utc=True)

In [19]:
df.sort_values("scrape_date",inplace=True)

In [35]:
df.loc[:,"last_updated"] = df["last_updated"].apply(lambda x: extract_datetime(x) if isinstance(x, str) else x)
df["last_updated"] = pd.to_datetime(df["last_updated"])

In [45]:
pd.api.types.is_float(df["last_updated"])

False

In [53]:
df["last_updated"].dtype

dtype('<M8[ns]')

180845    Nei
713579    NaN
603003    NaN
332579    NaN
202901    NaN
         ... 
406158    NaN
406157    NaN
406156    NaN
406174    NaN
152793    Nei
Name: liens, Length: 810735, dtype: object

In [47]:
pd.api.types.is_any_real_numeric_dtype(df['liens'])

False

# Ensure extra columns

In [18]:
add_if_missing = ['web','email','warranty','color_interior','gearbox_type','warranty_length',
                  'condition_report', ]
for col in add_if_missing:
    if col not in df.columns:
        df[col] = None

In [26]:
dtype_map = {
                'object': 'STRING',
                'string': 'STRING',
                'int64': 'INTEGER',
                'Int64': 'INTEGER',
                'int64[pyarrow]': 'INTEGER',
                'float64': 'FLOAT',
                'Float64': 'FLOAT',
                'bool': 'BOOLEAN',
                'decimal.Decimal' : "NUMERIC",
                'Decimal' : "NUMERIC",
                'boolean': 'BOOLEAN',
                'datetime64[ns]': 'DATETIME',
                'datetime64[ns, UTC]': 'DATETIME',
                'date32[day][pyarrow]': 'DATE',
                'datetime64[us]': 'DATETIME',
                'category': 'STRING',
                'str' : 'STRING',
                'list' : ("STRING", "REPEATED"),
                'datetime' : 'DATETIME',
                'Timestamp' : 'TIMESTAMP'
            }

In [33]:
d = bq._get_dtype(df,"dealer_rating")
d is None

2025-08-14 15:57:51,443 - cleanCars - WARNING - Column 'dealer_rating' contains only null values or is empty.


True

## Save to BQ

In [94]:
logger.debug(f'Length: {len(df)} | before saving to BQ. Replace {replace}')
if replace:
    bq.to_bq(df,
             table_name=dataset,
             dataset_name='clean',
             if_exists='replace')
else:
    bq.to_bq(df,
             table_name=dataset,
             dataset_name='clean',
             if_exists='merge',
             merge_on=['item_id'])

2025-07-09 14:00:59,345 - cleanCars - DEBUG - Length: 547870 | before saving to BQ. Replace True
2025-07-09 14:02:18,241 - cleanCars - INFO - 547870 rader lagret i sibr-market.clean.cars


## EDA

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 547870 entries, 0 to 547869
Columns: 132 entries, item_id to block_heater
dtypes: Float64(2), Int64(18), bool(51), boolean(2), datetime64[ns, UTC](1), datetime64[us](1), dbdate(2), object(55)
memory usage: 369.4+ MB


In [46]:
df.head()

,item_id,title,model_year,mileage,gearbox,fuel,transfer_fee,price_excl_transfer,first_registration,color,color_description,wheel_drive,power,engine_volume,co2,weight,seats,body_type,prev_owners,doors,cargo_space,state,sales_type,category,reg_num,vin,trailer_weight,last_eu,next_eu,range,battery,features,known_issues,major_repairs,engine_tuned,liens,address,last_updated,url,description,subtitle,dealer,contact_person,phone,email,total_price,warranty_until,scrape_date,country,web,dealer_rating,dealer_n_ratings,brand,model,color_interior,fuel_cons,length,height,width,condition,manufacturer,gearbox_type,warranty,warranty_length,emission_class,emission_sticker,condition_report,service_history,service_book,vat_deductible,rn,FIRST,LAST,salgstid,clean_date,postal_code,municipality,county,region,car_for_parts,timing_belt,cruise_control,parking_sensor_behind,parking_sensor_front,parking_assistant,driving_computer,rear_view_camera,bluetooth,tow_hitch,360_camera,internet,app,skin_interior,air_conditioning,dab,apple_carplay,TCS,ESC,sunroof,traffic_sign_recognition,led_lights,xenon_lights,alarm,high_beam_assistant,keyless_start,keyless_go,abs,isofix,roof_railing,roof_box,pollen_filter,airbag,power_steering,metallic_paint,handsfree,alloy_wheels_summer,alloy_wheels_winter,navigation,charging_cable,central_locking,rain_sensor,heated_seats,heated_steering_wheel,electric_tailgate,electric_mirrors,electric_windows,electric_seats,heated_windshield,winter_tires,summer_tires,multi_function_stearing_wheel,block_heater
0,410707921,Toyota HiLux,2002,324000,Manuell,diesel,1268,60000,4.1.2002,Grønn,Grønn metallic,Firehjulsdrift,102,2.5,<NA>,1745,2,Pickup,<NA>,2,<NA>,Norge,Bruktbil til salgs,Varebil,HF20960,JTFWS796200001436,2250,<NA>,<NA>,<NA>,<NA>,"[airbag foran, aux-inngang, cd-spiller, el.vin...",Ja. Sprukket frontrute/ ulyd fra bakaksel,False,Nei,False,8664 Mosjøen,"24. juni 2025, 11:36",https://www.finn.no/mobility/item/410707921,"['Ny pris!', 'Selger min hilux da behovet er e...",D-4D X-Cab ny pris,private,NaN,NaN,NaN,61268,NaN,2025-06-25 00:00:00+00:00,NO,NaN,NaN,NaN,toyota,HiLux,Sort,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2025-06-04,2025-06-25,21,2025-07-09 12:31:39.238340,8664,VEFSN,Nordland,Nord-Norge,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,True,False,False,True,True,False,False,True,True,False,False
1,393055153,Suzuki Swift,2023,5000,Manuell,el + bensin,4812,294188,25.9.2023,Grå,Mørk Grå metallic,Firehjulsdrift,83,1.2,<NA>,949,5,Kombi 5-dørs,<NA>,5,265.0,Norge,Bruktbil til salgs,Personbil,HB88035,JSAAZDA3S00509528,1000,<NA>,<NA>,<NA>,<NA>,"[aeb/fotgjenger, aeb/lav hastighet, antispinn,...",False,False,False,False,"Torolf Storsveens veg 12, 2406 Elverum","10. februar 2025, 14:43",https://www.finn.no/mobility/item/393055153,Denne bilen selges med Nybilgaranti (43 md. / ...,4X4 GLX HYBRID man Demobil/Vinterhjul,Slatlem Elverum AS,Slatlem Elverum AS,62 43 23 00,salg.elverum@slatlem.no,299000,100 000 km,2025-03-30 00:00:00+00:00,NO,None,None,None,suzuki,Swift,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,1,2025-02-11,2025-03-30,47,2025-07-09 12:31:39.238340,2406,ELVERUM,Innlandet,Østlandet,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False
2,415370785,Ford Ranger,2010,135000,Manuell,diesel,1268,98731,30.8.2010,Svart,NaN,Firehjulsdrift,143,2.5,255,1935,4,Pickup,<NA>,4,<NA>,Norge,Bruktbil til salgs,Personbil,RK19690,WF0UM2E109W849528,3000,<NA>,<NA>,<NA>,<NA>,"[abs-bremser, airbag foran, antiskrens, antisp...",False,False,False,False,"Ringer

bluetooth
True     351322
False    196548
Name: count, dtype: int64

In [37]:
models = cs.download("models.json", read_in_file=True)

2025-08-14 19:14:24,158 - cleanCars - INFO - Read in models.json


In [41]:
models_df = pd.DataFrame.from_dict(models)

In [42]:
models_df

,dataset,target,log_target,created_at,training_columns,params,r2_score,mse,mse_train,r2_train,filename,model_type
0,cars_el,total_price,True,1755198685504,"{'model_year': ['int64', 2023], 'mileage': ['i...","{'depth': 7, 'iterations': 1223, 'l2_leaf_reg'...",0.992941,2.608586e+08,1.700918e+08,0.995468,models/CatBoostRegressor_cars_el.pkl,CatBoostRegressor
1,cars_fossil,total_price,True,1755198737521,"{'model_year': ['int64', 2012], 'mileage': ['i...","{'depth': 4, 'iterations': 1463, 'l2_leaf_reg'...",0.991965,4.271395e+08,3.040118e+08,0.994292,models/CatBoostRegressor_cars_fossil.pkl,CatBoostRegressor
2,homes_apartments,price,True,1755198754894,"{'joint_debt': ['int64', 438841], 'monthly_com...",{'model__learning_rate': np.float64(0.02545642...,0.938795,6.023339e+11,4.886661e+11,0.950053,models/XGBRegressor_homes_apartments.pkl,XGBRegressor
3,homes_houses,price,True,1755198759145,"{'bedrooms': ['int64', 4], 'internal_area': ['...",{'model__learning_rate': np.float64(0.03233791...,0.876566,2.102727e+12,1.974589e+12,0.881561,models/XGBRegressor_homes_houses.pkl,XGBRegressor
4,homes_apartments_oslo,price,True,1755198761806,"{'joint_debt': ['int64', 115139], 'monthly_com...",{'model__learning_rate': np.float64(0.03750796...,0.932156,7.760061e+11,4.830825e+11,0.957538,models/XGBRegressor_homes_apartments_oslo.pkl,XGBRegressor
5,rentals,monthly_rent,True,1755198779555,"{'bedrooms': ['int64', 1], 'floor': ['int64', ...","{'model__depth': 7, 'model__iterations': 1223,...",0.847634,5.905503e+06,4.807642e+06,0.879095,models/CatBoostRegressor_rentals.pkl,CatBoostRegressor
6,rentals_oslo,monthly_rent,True,1755198786239,"{'bedrooms': ['int64', 2], 'floor': ['int64', ...","{'model__depth': 9, 'model__iterations': 809, ...",0.856547,6.921887e+06,4.354326e+06,0.907098,models/CatBoostRegressor_rentals_oslo.pkl,CatBoostRegressor
7,rentals_co-living,monthly_rent,True,1755198791503,"{'floor': ['int64', 0], 'eq_power_True': ['boo...","{'model__depth': 9, 'model__iterations': 809, ...",0.423107,1.661136e+06,1.425511e+06,0.625768,models/CatBoostRegressor_rentals_co-living.pkl,CatBoostRegressor
